Deploying SQL Server 2019 in Kubernetes

In [3]:
kubectl apply -f https://raw.githubusercontent.com/nocentino/Presentations/master/DataExposed/SQLServer.yaml

persistentvolumeclaim/pvc-sql-system unchanged
deployment.apps/mssql-deployment unchanged
service/mssql-deployment unchanged


Creating a database

In [ ]:
kubectl get service
$SERVICEIP=kubectl get service mssql-deployment -o jsonpath='{.status.loadBalancer.ingress[].hostname}'
$PORT=31433
Write-Output $SERVICEIP,$PORT


#Check out the servername and the version in the container, 2017 CU16...
sqlcmd -S $SERVICEIP,$PORT -U sa -Q "SELECT @@SERVERNAME,@@VERSION" -P 'S0methingS@Str0ng!' 

Crashing SQL Server

Updating SQL Server